# Get ability IDs

Used to map ability IDs to ability names. Should be ran sparingly/probably only after every major patch. This makes a lot of API calls and takes a while to run.

In [9]:
import json
import time
import requests
import os

# API config
url = "https://www.fflogs.com/api/v2/client"
api_key = os.environ['FFLOGS_TOKEN'] # or copy/paste your key here

headers = {
    'Content-Type': "application/json",
    'Authorization': f"Bearer {api_key}"
}

json_payload = {"query":"""
query abilityName{
    gameData{
        abilities(page:1){
            data{
                id
                name
            }
            last_page
        }
    }
}""",
"operationName":"abilityName"
}

r = requests.post(url=url, json=json_payload, headers=headers)
data = r.text

last_page = json.loads(data)['data']['gameData']['abilities']['last_page']
abilities = json.loads(data)['data']['gameData']['abilities']['data']

for p in range(2, last_page+1):
    json_payload = {
    "query": """
        query abilityName($page_number: Int!){
            gameData{
                abilities(page:$page_number){
                    data{
                        id
                        name
                    }
                    last_page
                }
            }
        }
        """,
    "variables": {"page_number": p},
    "operation_name": "abilityName"
    }

    r = requests.post(url=url, json=json_payload, headers=headers)
    data = r.text

    abilities.extend(json.loads(data)['data']['gameData']['abilities']['data'])

    if p % 10 == 0:
        print(f'On page {p}/{last_page}', end='\r')
    
    time.sleep(0.5)

abilities

[{'id': 1, 'name': 'Key Item'},
 {'id': 2, 'name': 'Interaction'},
 {'id': 3, 'name': 'Sprint'},
 {'id': 4, 'name': 'Mount'},
 {'id': 5, 'name': 'Teleport'},
 {'id': 6, 'name': 'Return'},
 {'id': 7, 'name': 'attack'},
 {'id': 8, 'name': 'Shot'},
 {'id': 9, 'name': 'Fast Blade'},
 {'id': 10, 'name': 'Rampart'},
 {'id': 11, 'name': 'Savage Blade'},
 {'id': 12, 'name': 'Convalescence'},
 {'id': 13, 'name': 'Awareness'},
 {'id': 14, 'name': 'Flash'},
 {'id': 15, 'name': 'Riot Blade'},
 {'id': 16, 'name': 'Shield Bash'},
 {'id': 17, 'name': 'Sentinel'},
 {'id': 18, 'name': 'Provoke'},
 {'id': 19, 'name': 'Tempered Will'},
 {'id': 20, 'name': 'Fight or Flight'},
 {'id': 21, 'name': 'Rage of Halone'},
 {'id': 22, 'name': 'Bulwark'},
 {'id': 23, 'name': 'Circle of Scorn'},
 {'id': 24, 'name': 'Shield Lob'},
 {'id': 25, 'name': 'Shield Swipe'},
 {'id': 26, 'name': 'Sword Oath'},
 {'id': 27, 'name': 'Cover'},
 {'id': 28, 'name': 'Iron Will'},
 {'id': 29, 'name': 'Spirits Within'},
 {'id': 30, 'n

In [25]:
abilities_dict = {}
for idx, d in enumerate(abilities):
    abilities_dict[d['id']] =  d['name']

# I forget why this is here, but I think it's important
abilities_dict[-1] = None

with open('abilities.json', 'w') as f:
    json.dump(abilities_dict, f, indent=2)